In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

df_game_events_dummy = pd.read_parquet("Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/transfermarkrt-dados-clean/dummy_data/game_events_dummy.parquet")
df_games = pd.read_parquet('Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/transfermarkrt-dados-clean/games.parquet')
df_club_games = pd.read_parquet("Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/transfermarkrt-dados-clean/club_games.parquet")
df_appereances = pd.read_parquet("Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/transfermarkrt-dados-clean/appearances.parquet")
merged_filtred = pd.read_parquet('Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/transfermarkrt-dados-clean/merge_filtred_default.parquet')
df_game_events = pd.read_parquet(r"Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/transfermarkrt-dados-clean/dummy_data/game_events_descr_dummy_filtred.parquet")

In [2]:
game_stats = df_appereances.groupby(['game_id', 'player_club_id'])[['yellow_cards', 'red_cards', 'goals', 'assists']].sum().reset_index()
game_stats.rename(columns={'player_club_id': 'club_id'}, inplace=True)
game_stats

,game_id,club_id,yellow_cards,red_cards,goals,assists
0,2262147,419,0,0,0,0
1,2262147,720,0,0,2,2
2,2262148,338,0,0,1,1
3,2262148,583,1,0,4,3
4,2262149,11,1,0,2,2
...,...,...,...,...,...,...
2875,4331177,418,1,0,2,1
2876,4331178,27,2,0,2,2
2877,4331178,418,2,0,2,2
2878,4343512,16,3,0,0,0


In [3]:
df_games = df_games[['game_id', 'home_club_id', 'away_club_id', 'home_club_name', 'away_club_name', 'home_club_goals', 'away_club_goals', 'home_club_formation', 'away_club_formation']]
df_games

,game_id,home_club_id,away_club_id,home_club_name,away_club_name,home_club_goals,away_club_goals,home_club_formation,away_club_formation
105,2368466,371,5,celtic fc,AC Milan,0,3,4-2-3-1,4-3-2-1
106,2368481,683,58,olympiacos piraeus,RSC Anderlecht,3,1,4-4-1-1,4-1-4-1
111,2384520,141,506,galatasaray,Juventus FC,1,0,3-5-2,3-5-2 flat
212,2490853,610,583,ajax amsterdam,Paris Saint-Germain,1,1,4-3-3 Attacking,4-3-3 Attacking
222,2495311,12,27,as roma,Bayern Munich,1,7,4-3-3 Attacking,3-5-2
...,...,...,...,...,...,...,...,...,...
68274,3651124,2481,418,None,Real Madrid,0,3,4-2-3-1,4-3-3 Attacking
68294,4243700,16,383,borussia dortmund,PSV Eindhoven,2,0,4-2-3-1,4-3-3 Defending
68304,3651114,1082,409,losc lille,None,1,0,4-4-2 double 6,4-3-1-2
68312,3098025,1049,452,valencia cf,None,3,1,4-4-2 double 6,4-4-2 double 6


# Junção dos datasets
Aqui realizamos a junção dos dataset que jugamos relevantes para a analise

Criou uma linha para cada time que em cada uma das partidas de modo que fique correta a disposição dos dados

In [4]:
# Cria um DataFrame para os times da casa
home_teams = df_games[['game_id', 'home_club_id', 'home_club_name', 'home_club_goals', 'home_club_formation']].copy()
home_teams['team_type'] = 'home'
home_teams = home_teams.rename(columns={
    'home_club_id': 'club_id',
    'home_club_name': 'club_name',
    'home_club_goals': 'club_goals',
    'home_club_formation': 'club_formation'
})

# Cria um DataFrame para os times visitantes
away_teams = df_games[['game_id', 'away_club_id', 'away_club_name', 'away_club_goals', 'away_club_formation']].copy()
away_teams['team_type'] = 'away'
away_teams = away_teams.rename(columns={
    'away_club_id': 'club_id',
    'away_club_name': 'club_name',
    'away_club_goals': 'club_goals',
    'away_club_formation': 'club_formation'
})

# Concatena os DataFrames
games_sep_times = pd.concat([home_teams, away_teams])

# Ordena pelo 'game_id'
games_sep_times = games_sep_times.sort_values('game_id')

In [5]:
import numpy as np

games_sep_times['club_formation'] = games_sep_times['club_formation'].replace('None', np.nan)
games_sep_times = games_sep_times[games_sep_times['club_formation'].notna()]
games_sep_times

,game_id,club_id,club_name,club_goals,club_formation,team_type
66271,2367027,15,Bayer 04 Leverkusen,2,4-3-3 Attacking,away
66271,2367027,985,manchester united,4,4-4-1-1,home
37051,2367028,681,real sociedad,0,4-3-3 Attacking,home
37051,2367028,660,Shakhtar Donetsk,2,4-2-3-1,away
48243,2367029,141,galatasaray,1,4-4-2 Diamond,home
...,...,...,...,...,...,...
20026,4331177,27,Bayern Munich,1,4-2-3-1,away
8513,4331178,27,bayern munich,2,4-2-3-1,home
8513,4331178,418,Real Madrid,2,4-4-2 double 6,away
51994,4343512,16,borussia dortmund,0,4-3-3 Attacking,home


In [6]:
games_sep_times['club_formation'].value_counts()

club_formation
4-2-3-1            788
4-3-3 Attacking    718
4-4-2 double 6     269
4-1-4-1            157
3-5-2 flat         142
3-4-2-1            123
4-3-1-2             90
4-4-2               87
4-3-3 Defending     57
3-4-3               52
5-4-1               48
5-3-2               37
3-4-1-2             37
4-4-1-1             33
3-5-2               28
4-4-2 Diamond       15
4-1-3-2             12
3-5-2 Attacking     12
4-5-1 flat          10
4-3-2-1              8
3-1-4-2              7
3-4-3 Diamond        3
4-5-1                3
3-3-3-1              2
Name: count, dtype: int64

In [7]:
games_sep_times

,game_id,club_id,club_name,club_goals,club_formation,team_type
66271,2367027,15,Bayer 04 Leverkusen,2,4-3-3 Attacking,away
66271,2367027,985,manchester united,4,4-4-1-1,home
37051,2367028,681,real sociedad,0,4-3-3 Attacking,home
37051,2367028,660,Shakhtar Donetsk,2,4-2-3-1,away
48243,2367029,141,galatasaray,1,4-4-2 Diamond,home
...,...,...,...,...,...,...
20026,4331177,27,Bayern Munich,1,4-2-3-1,away
8513,4331178,27,bayern munich,2,4-2-3-1,home
8513,4331178,418,Real Madrid,2,4-4-2 double 6,away
51994,4343512,16,borussia dortmund,0,4-3-3 Attacking,home


In [8]:
game_stats

,game_id,club_id,yellow_cards,red_cards,goals,assists
0,2262147,419,0,0,0,0
1,2262147,720,0,0,2,2
2,2262148,338,0,0,1,1
3,2262148,583,1,0,4,3
4,2262149,11,1,0,2,2
...,...,...,...,...,...,...
2875,4331177,418,1,0,2,1
2876,4331178,27,2,0,2,2
2877,4331178,418,2,0,2,2
2878,4343512,16,3,0,0,0


In [9]:
merge_game_sep_stats = games_sep_times.merge(game_stats, on=['club_id', 'game_id'])
merge_game_sep_stats_filtred = merge_game_sep_stats[['club_id', 'game_id','club_name', 'club_formation', 'team_type', 'yellow_cards', 'red_cards', 'goals', 'assists']]

In [10]:
merge_game_sep_stats_filtred['club_name'] = merge_game_sep_stats_filtred['club_name'].str.lower()

C:\Users\Eduar\AppData\Local\Temp\ipykernel_23148\640653327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_game_sep_stats_filtred['club_name'] = merge_game_sep_stats_filtred['club_name'].str.lower()


In [11]:
stats = merge_game_sep_stats_filtred[['yellow_cards', 'red_cards', 'goals', 'assists']]

cols_to_dummy = merge_game_sep_stats_filtred.drop(columns=['club_id', 'game_id', 'yellow_cards', 'red_cards', 'goals', 'assists'])

cols_dummified = pd.get_dummies(cols_to_dummy)
cols_dummified = cols_dummified.astype(int)

merge_game_sep_stats_filtred_dummy = pd.concat([stats, cols_dummified], axis=1)


In [12]:
merge_game_sep_stats_filtred['club_name'] = merge_game_sep_stats_filtred['club_name'].str.lower()

C:\Users\Eduar\AppData\Local\Temp\ipykernel_23148\640653327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_game_sep_stats_filtred['club_name'] = merge_game_sep_stats_filtred['club_name'].str.lower()


In [13]:
from sklearn.preprocessing import MinMaxScaler

# Select the columns to be scaled
columns_to_scale = ['yellow_cards', 'red_cards', 'goals', 'assists']

# Create a MinMaxScaler object
scaler = MinMaxScaler()

merge_game_sep_stats_filtred_normalized = merge_game_sep_stats_filtred.copy()

# Apply MinMaxScaler to the selected columns
merge_game_sep_stats_filtred_normalized[columns_to_scale] = scaler.fit_transform(merge_game_sep_stats_filtred[columns_to_scale])

# Display the updated DataFrame
merge_game_sep_stats_filtred_normalized

,club_id,game_id,club_name,club_formation,team_type,yellow_cards,red_cards,goals,assists
0,15,2367027,bayer 04 leverkusen,4-3-3 Attacking,away,0.250,0.0,0.250,0.250
1,985,2367027,manchester united,4-4-1-1,home,0.125,0.0,0.500,0.500
2,681,2367028,real sociedad,4-3-3 Attacking,home,0.250,0.0,0.000,0.000
3,660,2367028,shakhtar donetsk,4-2-3-1,away,0.625,0.0,0.250,0.250
4,141,2367029,galatasaray,4-4-2 Diamond,home,0.375,0.0,0.125,0.125
...,...,...,...,...,...,...,...,...,...
2626,27,4331177,bayern munich,4-2-3-1,away,0.000,0.0,0.125,0.125
2627,27,4331178,bayern munich,4-2-3-1,home,0.250,0.0,0.250,0.250
2628,418,4331178,real madrid,4-4-2 double 6,away,0.250,0.0,0.250,0.250
2629,16,4343512,borussia dortmund,4-3-3 Attacking,home,0.375,0.0,0.000,0.000


In [14]:
stats = merge_game_sep_stats_filtred_normalized[['yellow_cards', 'red_cards', 'goals', 'assists']]

cols_to_dummy = merge_game_sep_stats_filtred_normalized.drop(columns=['club_id', 'game_id','yellow_cards', 'red_cards', 'goals', 'assists'])

cols_dummified = pd.get_dummies(cols_to_dummy)
cols_dummified = cols_dummified.astype(int)

merge_game_sep_stats_filtred_normalized_dummy = pd.concat([stats, cols_dummified], axis=1)

In [15]:
from sklearn.preprocessing import StandardScaler

# Selecionar as colunas a serem padronizadas
columns_to_standardize = ['yellow_cards', 'red_cards', 'goals', 'assists']

# Criar um objeto StandardScaler
scaler = StandardScaler()

# Aplicar a padronização nas colunas selecionadas
merge_game_sep_stats_filtred_standardized = merge_game_sep_stats_filtred.copy()
merge_game_sep_stats_filtred_standardized[columns_to_standardize] = scaler.fit_transform(merge_game_sep_stats_filtred[columns_to_standardize])

# Exibir o DataFrame atualizado
merge_game_sep_stats_filtred_standardized

,club_id,game_id,club_name,club_formation,team_type,yellow_cards,red_cards,goals,assists
0,15,2367027,bayer 04 leverkusen,4-3-3 Attacking,away,0.079934,-0.184976,0.423299,0.659087
1,985,2367027,manchester united,4-4-1-1,home,-0.616447,-0.184976,1.872483,2.263955
2,681,2367028,real sociedad,4-3-3 Attacking,home,0.079934,-0.184976,-1.025886,-0.945780
3,660,2367028,shakhtar donetsk,4-2-3-1,away,2.169078,-0.184976,0.423299,0.659087
4,141,2367029,galatasaray,4-4-2 Diamond,home,0.776316,-0.184976,-0.301294,-0.143346
...,...,...,...,...,...,...,...,...,...
2626,27,4331177,bayern munich,4-2-3-1,away,-1.312828,-0.184976,-0.301294,-0.143346
2627,27,4331178,bayern munich,4-2-3-1,home,0.079934,-0.184976,0.423299,0.659087
2628,418,4331178,real madrid,4-4-2 double 6,away,0.079934,-0.184976,0.423299,0.659087
2629,16,4343512,borussia dortmund,4-3-3 Attacking,home,0.776316,-0.184976,-1.025886,-0.945780


In [16]:
stats = merge_game_sep_stats_filtred_standardized[['yellow_cards', 'red_cards', 'goals', 'assists']]

cols_to_dummy = merge_game_sep_stats_filtred_standardized.drop(columns=['club_id', 'game_id','yellow_cards', 'red_cards', 'goals', 'assists'])

cols_dummified = pd.get_dummies(cols_to_dummy)
cols_dummified = cols_dummified.astype(int)

merge_game_sep_stats_filtred_standardized_dummy = pd.concat([stats, cols_dummified], axis=1)

In [17]:
stats = merge_game_sep_stats_filtred[['yellow_cards', 'red_cards', 'goals', 'assists']]

cols_to_dummy = merge_game_sep_stats_filtred_normalized.drop(columns=['club_id', 'game_id','yellow_cards', 'red_cards', 'goals', 'assists'])

cols_dummified = pd.get_dummies(cols_to_dummy)
cols_dummified = cols_dummified.astype(int)

merge_game_sep_stats_filtred_dummy = pd.concat([stats, cols_dummified], axis=1)

In [18]:
merge_game_sep_stats_filtred = merge_game_sep_stats[['club_name', 'club_formation', 'team_type', 'yellow_cards', 'red_cards', 'goals', 'assists']]

# Salvando tabelas na pasta data
1. Merge_Filtred.parquet
2. Merge_Normalized_Dummy.parquet
3. Merge_Standardized_Dummy.parquet

In [19]:
merge_game_sep_stats_filtred.to_parquet("Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/clustering/data/merge_filtred_default.parquet")
merge_game_sep_stats_filtred_dummy.to_parquet("Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/clustering/data/merge_filtred_dummy.parquet")
merge_game_sep_stats_filtred_normalized_dummy.to_parquet("Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/clustering/data/merge_normalized_dummy.parquet")
merge_game_sep_stats_filtred_standardized_dummy.to_parquet("Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/clustering/data/merge_standardized_dummy.parquet")